In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

import os
import re
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

In [14]:
acts = '../../thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet'
entities = '../../thais/contrato-licitações/estudo_crf/result/labelbox_entidades.parquet'
texts = '../../thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet'

df1 = pd.read_parquet(acts)
df2 = pd.read_parquet(entities)
df3 = pd.read_parquet(texts)

In [15]:
df1

,arquivo_rast,text,ato,dodf,treated_text
0,0_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,aviso_de_licitacao,DODF_010_15-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...
1,0_aviso_de_licitacao_DODF_093_20-05-2019.txt,xxbcet AVISO DE LICITAÇÃO xxecet\r\nxxbcet PRE...,aviso_de_licitacao,DODF_093_20-05-2019,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...
2,2_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet PREGÃO ELETRÔNICO Nº 14/2021 xxecet\nPr...,aviso_de_licitacao,DODF_010_15-01-2021,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...
3,6_aviso_de_licitacao_DODF_002_05-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,aviso_de_licitacao,DODF_002_05-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...
4,9_extrato_de_contrato_ou_convenio_DODF_004_07-...,xxbcet EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2...,extrato_de_contrato_ou_convenio,DODF_004_07-01-2016,EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2015 Esp...
...,...,...,...,...,...
1163,100_rinst_extrato_de_aditamento_contratual_DOD...,xxbcet EXTRATO DE INSTRUMENTO CONTRATUAL xxece...,aviso_de_aditamento_contratual,DODF_189_02-10-2017,EXTRATO DE INSTRUMENTO CONTRATUAL PROCESSO: 11...
1164,101_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.003.048/2016. ESPÉCIE: SEXTO TER...,aviso_de_aditamento_contratual,DODF_189_02-10-2017,PROCESSO: 112.003.048/2016. ESPÉCIE: SEXTO TER...
1165,102_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.003.777/2013. ESPÉCIE: NONO TERM...,aviso_de_aditamento_contratual,DODF_189_02-10-2017,PROCESSO: 112.003.777/2013. ESPÉCIE: NONO TERM...
1166,103_rinst_extrato_de_aditamento_contratual_DOD...,PROCESSO: 112.004.652/2015. ESPÉCIE: SEGUNDO T...,aviso_de_aditamento_contratual,DODF_189_02-10-2017,PROCESSO: 112.004.652/2015. ESPÉCIE: SEGUNDO T...


In [16]:
df2

,titulo_entidade,valor_entidade,entidade,arquivo_rast,loc_begin,loc_end,ato,ner_token
0,Processo do GDF,processo_do_gdf,150.001.430/2010,99_extrato_de_contrato_ou_convenio_DODF_179_17...,59,75,extrato_de_contrato_ou_convenio,PROCESSO
1,Número do ajuste,numero_do_ajuste,292/2010,99_extrato_de_contrato_ou_convenio_DODF_179_17...,30,38,extrato_de_contrato_ou_convenio,NUM_AJUSTE
2,Órgão contratante,orgao_contratante,DF/SC/FAC,99_extrato_de_contrato_ou_convenio_DODF_179_17...,85,94,extrato_de_contrato_ou_convenio,CONTRATANTE
3,Entidade contratada,entidade_contratada,PATRÍCIA BARROSO HERZOG,99_extrato_de_contrato_ou_convenio_DODF_179_17...,97,120,extrato_de_contrato_ou_convenio,CONTRATADA
4,Objeto do ajuste,objeto_do_ajuste,O presente\r\r\ncontrato tem por objeto o fome...,99_extrato_de_contrato_ou_convenio_DODF_179_17...,193,384,extrato_de_contrato_ou_convenio,OBJ_AJUSTE
...,...,...,...,...,...,...,...,...
7941,Objeto do aditamento contratual,objeto_do_aditamento_contratual,O objeto do presente Termo Aditivo é a convali...,103_rinst_extrato_de_aditamento_contratual_DOD...,297,993,aviso_de_aditamento_contratual,OBJ_ADITIVO
7942,Órgão contratante,orgao_contratante,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,104_rinst_extrato_de_aditamento_contratual_DOD...,153,210,aviso_de_aditamento_contratual,CONTRATANTE
7943,Número do contrato,numero_do_contrato,610/2014,104_rinst_extrato_de_aditamento_contratual_DOD...,116,124,aviso_de_aditamento_contratual,NUM_CONTRATO
7944,Número do termo aditivo,numero_do_termo_aditivo,SÉTIMO TERMO ADITIVO,104_rinst_extrato_de_aditamento_contratual_DOD...,40,60,aviso_de_aditamento_contratual,NUM_ADITIVO


In [17]:
df2['ner_token'].unique()

array(['PROCESSO', 'NUM_AJUSTE', 'CONTRATANTE', 'CONTRATADA',
       'OBJ_AJUSTE', 'VIGENCIA', 'VALOR', 'PT', 'DATA_ASSINATURA',
       'CODIGO_UO', 'ND', 'NE', 'FUND_DISPENSA', 'ORGAO_LICITANTE',
       'NUM_LICITACAO', 'OBJ_LICITACAO', 'MODALIDADE_LICITACAO',
       'DATA_ABERTURA', 'TIPO_OBJ', 'VALOR_ESTIMADO',
       'CODIGO_SISTEMA_COMPRAS', 'NUM_CONTRATO', 'OBJ_ADITIVO',
       'NUM_ADITIVO', 'IDENT_REVOGACAO_ANULACAO', 'IDENT_DISPENSA',
       'SISTEMA_COMPRAS', 'CONVENENTE', 'DECISAO_TCDF', 'PRAZO_SUSPENSAO',
       'OG_ATA'], dtype=object)

In [18]:
tokens = df2['ner_token'].unique()

for token in tokens:
    df1[token] = np.nan

In [19]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [20]:
for i in range(len(df2)):
    for j in range(len(df1)):
        if df2['arquivo_rast'][i] == df1['arquivo_rast'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(df2['entidade'][i]).replace("\r", ""))
            df1.loc[j, df2['ner_token'][i]] = entitie


### IOB

In [21]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [22]:
df1.columns

Index(['arquivo_rast', 'text', 'ato', 'dodf', 'treated_text', 'PROCESSO',
       'NUM_AJUSTE', 'CONTRATANTE', 'CONTRATADA', 'OBJ_AJUSTE', 'VIGENCIA',
       'VALOR', 'PT', 'DATA_ASSINATURA', 'CODIGO_UO', 'ND', 'NE',
       'FUND_DISPENSA', 'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'DATA_ABERTURA', 'TIPO_OBJ', 'VALOR_ESTIMADO',
       'CODIGO_SISTEMA_COMPRAS', 'NUM_CONTRATO', 'OBJ_ADITIVO', 'NUM_ADITIVO',
       'IDENT_REVOGACAO_ANULACAO', 'IDENT_DISPENSA', 'SISTEMA_COMPRAS',
       'CONVENENTE', 'DECISAO_TCDF', 'PRAZO_SUSPENSAO', 'OG_ATA'],
      dtype='object')

In [26]:
# df1.to_parquet('all_acts_entities.parquet')

In [14]:
data = df1[['PROCESSO', 'NUM_AJUSTE', 'CONTRATANTE', 'CONTRATADA', 'OBJ_AJUSTE', 'VIGENCIA',
'VALOR', 'PT', 'DATA_ASSINATURA', 'CODIGO_UO', 'ND', 'NE',
'FUND_DISPENSA', 'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
'MODALIDADE_LICITACAO', 'VALOR_ESTIMADO',
'CODIGO_SISTEMA_COMPRAS', 'NUM_CONTRATO', 'OBJ_ADITIVO', 'NUM_ADITIVO',
'IDENT_REVOGACAO_ANULACAO', 'IDENT_DISPENSA', 'SISTEMA_COMPRAS',
'CONVENENTE', 'DECISAO_TCDF', 'PRAZO_SUSPENSAO', 'OG_ATA', 'treated_text',]]

In [15]:
data.drop(758, inplace=True)
data = data.reset_index(drop=True)

C:\Users\ianfi\AppData\Local\Temp\ipykernel_22752\291838482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(758, inplace=True)


In [16]:
iob = IOBifyer(column='treated_text')
r = iob.transform(data)

In [17]:
r

0       [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
1       [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
2       [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
3       [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
4       [O, O, O, O, O, B-NUM_AJUSTE, I-NUM_AJUSTE, O,...
                              ...                        
1162    [O, O, O, O, O, O, O, O, O, O, B-NUM_ADITIVO, ...
1163    [O, O, O, O, O, O, B-NUM_ADITIVO, I-NUM_ADITIV...
1164    [O, O, O, O, O, O, B-NUM_ADITIVO, I-NUM_ADITIV...
1165    [O, O, O, O, O, O, B-NUM_ADITIVO, I-NUM_ADITIV...
1166    [O, O, O, O, O, O, O, B-NUM_ADITIVO, I-NUM_ADI...
Length: 1167, dtype: object

In [18]:
data.shape, r.shape

((1167, 30), (1167,))

In [19]:
data["IOB"] = np.nan

In [20]:
for i in range(len(data)):
    data.loc[i, "IOB"] = ' '.join(r[i])

In [21]:
data['treated_text'][0]

'AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/2021 Processo: 00052-00021170/2020-47. OBJETO: Contratação de Concessionária Autorizada pela Montadora RENA ULT, localizada no Distrito Federal, para prestação de serviços de forma contínua durante o período de garantia, para as manutenções preventivas e corretivas, com fomecimento e aplicação de peças e acessórios genuínos, lubrificantes, óleos, aditivos, higienizadores, alinhamento e balanceamento para 40 (quarenta) viaturas marca RENAULT, modelo Importado Logan ZEN 16MT, anos/modelos 2020/2021, 1.6 flex (gasolina/álcool), pertencentes à frota da Polícia Civil do Distrito Federal., conforme especificações e condições estabelecidas no termo de referência constante do Anexo I do Edital. TIPO: Menor Preço. Valor estimado da licitação: R$ 826.897,20 (oitocentos e vinte e seis mil, oitocentos e noventa e sete reais e vinte centavos). Natureza de Despesa: 33.90.30 e 33.90.39, Fonte 100. Programa de Trabalho 28.845.0903.00NR.0053 Manutenção da Pol

In [23]:
len(DEFAULT_TOKENIZER(data['treated_text'][0]))

273

In [24]:
data['IOB'][0]

'O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM_LICITACAO O B-PROCESSO I-PROCESSO I-PROCESSO I-PROCESSO O B-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I

In [25]:
len(data['IOB'][0].split())

273

In [26]:
df_iob = data[['treated_text', 'IOB']]

In [28]:
df_iob.to_parquet('iob_all_acts.parquet')

In [8]:
df = pd.read_parquet('iob_all_acts.parquet')
df

,treated_text,IOB
0,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
3,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
4,EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2015 Esp...,O O O O O B-NUM_AJUSTE I-NUM_AJUSTE O O O O O ...
...,...,...
1162,EXTRATO DE INSTRUMENTO CONTRATUAL PROCESSO: 11...,O O O O O O O O O O B-NUM_ADITIVO I-NUM_ADITIV...
1163,PROCESSO: 112.003.048/2016. ESPÉCIE: SEXTO TER...,O O O O O O B-NUM_ADITIVO I-NUM_ADITIVO I-NUM_...
1164,PROCESSO: 112.003.777/2013. ESPÉCIE: NONO TERM...,O O O O O O B-NUM_ADITIVO I-NUM_ADITIVO I-NUM_...
1165,PROCESSO: 112.004.652/2015. ESPÉCIE: SEGUNDO T...,O O O O O O B-NUM_ADITIVO I-NUM_ADITIVO I-NUM_...


In [33]:
df['IOB'][0]

'O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM_LICITACAO O B-PROCESSO I-PROCESSO I-PROCESSO I-PROCESSO O B-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I